## Observations and Insights 

In [1]:
# 1 Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
mouse_metadata = pd.DataFrame(mouse_metadata)
mouse_metadata.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [2]:
len(mouse_metadata)

249

In [3]:
study_results = pd.read_csv(study_results_path)
study_results = pd.DataFrame(study_results)
study_results.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [4]:
len(study_results)

1893

In [5]:
# 2 Combine the data into a single dataset
# concatenate?
# frames = [mouse_metadata, study_results]
# merged_df = pd.concat(frames)

# or append?
# merged_df = mouse_metadata.append(study_results)

# or join?
# merged_df = pd.DataFrame.join(mouse_metadata, study_results)

# or merge?
merged_df = mouse_metadata.merge(study_results)

merged_df = pd.DataFrame(merged_df)
# Display the data table for preview
merged_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [6]:
print(merged_df.dtypes)

Mouse ID               object
Drug Regimen           object
Sex                    object
Age_months              int64
Weight (g)              int64
Timepoint               int64
Tumor Volume (mm3)    float64
Metastatic Sites        int64
dtype: object


In [7]:
# turn NaN to ""
merged_df = merged_df.fillna({"Drug Regimen": ""})
merged_df = merged_df.fillna({"Sex": ""})
merged_df = merged_df.fillna({"Age_months": ""})
merged_df = merged_df.fillna({"Weight (g)": ""})
merged_df = merged_df.fillna({"Timepoint": ""})
merged_df = merged_df.fillna({"Tumor Volume (mm3)": ""})
merged_df = merged_df.fillna({"Metastatic Sites": ""})

# change dtype to numeric
merged_df[["Age_months"]] = merged_df[["Age_months"]].apply(pd.to_numeric, downcast = "integer")
merged_df[["Weight (g)"]] = merged_df[["Weight (g)"]].apply(pd.to_numeric, downcast = "integer")
merged_df[["Timepoint"]] = merged_df[["Timepoint"]].apply(pd.to_numeric, downcast = "integer")
merged_df[["Tumor Volume (mm3)"]] = merged_df[["Tumor Volume (mm3)"]].apply(pd.to_numeric)
merged_df[["Metastatic Sites"]] = merged_df[["Metastatic Sites"]].apply(pd.to_numeric, downcast = "integer")
# Why doesn't downcast as integer work?
# merged_df.head()

In [8]:
print(merged_df.dtypes)

Mouse ID               object
Drug Regimen           object
Sex                    object
Age_months               int8
Weight (g)               int8
Timepoint                int8
Tumor Volume (mm3)    float64
Metastatic Sites         int8
dtype: object


In [9]:
# Checking the number of mice in the studies:
miceCount = merged_df.groupby(["Mouse ID"]).count()["Age_months"]
# miceCount

n = len(miceCount)
n

249

In [33]:
# Checking what drug regimens were studied
regimens = merged_df.groupby("Drug Regimen").count()
regimens

,Mouse ID,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
Drug Regimen,,,,,,,
Capomulin,230,230,230,230,230,230,230
Ceftamin,178,178,178,178,178,178,178
Infubinol,178,178,178,178,178,178,178
Ketapril,188,188,188,188,188,188,188
Naftisol,186,186,186,186,186,186,186
Placebo,181,181,181,181,181,181,181
Propriva,161,161,161,161,161,161,161
Ramicane,228,228,228,228,228,228,228
Stelasyn,181,181,181,181,181,181,181


In [34]:
# How many regimens were there?
regimenCount = len(regimens)
regimenCount

10

In [35]:
# This turns Drug Regimen from the index to a column of values...:
regimens = pd.DataFrame(regimens.index.tolist(), columns=["Drug Regimen"])
regimens

,Drug Regimen
0,Capomulin
1,Ceftamin
2,Infubinol
3,Ketapril
4,Naftisol
5,Placebo
6,Propriva
7,Ramicane
8,Stelasyn
9,Zoniferol


In [36]:
# ...and this reads those values into a list:
regimens = regimens["Drug Regimen"].tolist()
regimens

['Capomulin',
 'Ceftamin',
 'Infubinol',
 'Ketapril',
 'Naftisol',
 'Placebo',
 'Propriva',
 'Ramicane',
 'Stelasyn',
 'Zoniferol']

In [37]:
# 3a Getting the duplicate mice by ID number
# that shows up for Mouse ID and Timepoint. 

# I don't understand--I don't see Mouse ID in Timepoint.

In [38]:
# 3b Optional: Get all the data for the duplicate mouse ID. 


In [39]:
# 3c Create a clean DataFrame by dropping the duplicate mouse by its ID.


In [40]:
# 3d Checking the number of mice in the clean DataFrame.

## Summary Statistics

In [41]:
# 4 Generate a summary statistics table of mean, median, variance,
# standard deviation, and SEM of the tumor volume for each regimen
# miceCount.describe()

In [42]:
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

In [43]:
merged_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [44]:
tumorVolMean = merged_df.groupby(["Drug Regimen"]).mean()["Tumor Volume (mm3)"]
tumorVolMed = merged_df.groupby(["Drug Regimen"]).median()["Tumor Volume (mm3)"]
tumorVolVar = merged_df.groupby(["Drug Regimen"]).var()["Tumor Volume (mm3)"]
tumorVolStdDev = merged_df.groupby(["Drug Regimen"]).std()["Tumor Volume (mm3)"]
tumorVolSEM = merged_df.groupby(["Drug Regimen"]).sem()["Tumor Volume (mm3)"]

summary_df_verbose = pd.DataFrame({
    "Drug Regimen" : regimens,
    "Mean" : tumorVolMean,
     "Median" : tumorVolMed,
     "Variance" : tumorVolVar,
     "Std Deviation" : tumorVolStdDev,
     "Std Error of the Mean" : tumorVolSEM    
    })

#### Tumor Volume (mm3) by regimen

In [45]:
# 5 Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
summary_df_verbose

,Drug Regimen,Mean,Median,Variance,Std Deviation,Std Error of the Mean
Drug Regimen,,,,,,
Capomulin,Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,Propriva,52.322552,50.854632,42.351070,6.507770,0.512884
Ramicane,Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [69]:
# 6 Using the aggregation method, produce the same summary statistics in a single line
# @TA Farshad: "You are allowed to use group by in the 2nd method.
# The instructions is asking to create a summary table with a single line of code
# (versus creating a series each for mean, median, var, etc. and then creating a data frame).
# Please research pandas GroupBy.aggregate function.
# This is the URL of the current pandas documentation on GroupBy.aggregate:
# https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html?highlight=groupby%20aggregate#pandas.core.groupby.DataFrameGroupBy.aggregate
# which mentions Different aggregations per column
# df.groupby('A').agg({
# 'B': ['min', 'max'],
#     'C': 'sum'
#    })

#    B             C
#  min max       sum
# A
# 1   1   2  0.590715
# 2   3   4  0.704907

summary_df_concise = merged_df.groupby("Drug Regimen").agg({
     "Tumor Volume (mm3)" : ["mean"],
#    "Tumor Volume (mm3)" : ["median"],
#    "Tumor Volume (mm3)" : ["var"],
#    "Tumor Volume (mm3)" : ["stddev"],
#    "Tumor Volume (mm3)" : ["sem"]
    })
summary_df_concise

,Tumor Volume (mm3)
,mean
Drug Regimen,
Capomulin,40.675741
Ceftamin,52.591172
Infubinol,52.884795
Ketapril,55.235638
Naftisol,54.331565
Placebo,54.033581
Propriva,52.322552
Ramicane,40.216745


## Bar and Pie Charts

In [ ]:
# 7 Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [ ]:
# 8 Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [ ]:
# 9 Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# 10 Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# 11 Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# 12 Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# 13 Put treatments into a list for for loop (and later for plot labels)


# 14 Create empty list to fill with tumor vol data (for plotting)


# 15 Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# 16 Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# 17 Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# 18 Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [ ]:
# 19 Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
